In [6]:
from bs4 import BeautifulSoup
import urllib.request
url = 'http://www.yes24.com/Product/Search?domain=ALL&query=%ED%81%AC%EB%A1%A4%EB%A7%81'
html = urllib.request.urlopen(url)
soup = BeautifulSoup(html,'html.parser')
litag_list = soup.select('#yesSchList > li')
for temp in litag_list:
  print(temp.select('div > div.item_info > div.info_row.info_name > a.gd_name')[0].string)

만들면서 배우는 파이썬과 40개의 작품들
웹 크롤링 & 데이터 분석 with 파이썬
웹 크롤링 & 데이터 분석 with 파이썬
한입에 웹 크롤링
R을 이용한 웹 크롤링과 텍스트 분석
한입에 웹 크롤링
크롤링 핵심 가이드
파이썬을 이용한 웹크롤링과 데이터 분석
한번에 크롤링하는 고령친화정책론
웹 크롤링하는 한과장
한번에 크롤링하는 행정학
[로지텍코리아] 초고속 스크롤링 멀티 디바이스 무선 마우스 (M720 Triathlon)/블루투스,유니파잉
일잘러의 비밀, 구글 스프레드시트 제대로 파헤치기
Do it! 파이썬 생활 프로그래밍
칼퇴하는 일잘러의 업무 스킬, 파이썬 업무 자동화
파이썬 해킹 레시피
Must Have 코로나보드로 배우는 실전 웹 서비스 개발
당신의 칼퇴를 도와주는 UiPath 업무 자동화
クロ-リングハック 
R을 이용한 퀀트 투자 포트폴리오 만들기
Hello IT 파이썬을 제대로 활용해보려고 해
금융 데이터를 위한 파이썬
파이썬을 이용한 머신러닝, 딥러닝 실전 개발 입문
파이썬 자동화 교과서
